# Übung 2 Detektion von Verkehrsschildern

**German Traffic Sign Detection Benchmark**

Detallierte Beschreibung des Datensatzes siehe unter folgendem [Link](http://benchmark.ini.rub.de/?section=gtsdb&subsection=news)

## Imports

In [1]:
import os

import csv
import wget
import cv2
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display
from ipywidgets import interact, interactive, fixed, interact_manual, widgets

In [2]:
# Testfunktion für ipywidgets: 
# Es soll ein Slider angezeigt werden. Der Wertebereich des Sliders
# soll zwischen -10(min) und 30(max) liegen. 
# # Entsprechend der Sliderposition soll ein Ergebniswert angezeigt werden.
# def f(x):
#     return 3 * x
# interact(f, x= 10);

## Globale Variablen

Um hartcodierte Bezeichner/Namen in den Funktionen zu vermeiden, definiere an dieser Stelle alle Variablen, die global verwendet werden.

In [3]:
# Definiere den Pfad zum heruntergeladenen Datenordner
DATA_PATH = "./../data/FullIJCNN2013/"
# Prüfe, ob der Pfad existiert / korrekt eingegeben wurde
assert os.path.exists(DATA_PATH), "Der angegebene Pfad existriert nicht."

In [4]:
# Definiere den Pfad zur Datei gt.txt
"""
TIPP: arbeite zuerst mit der Datei new_gt.txt, um den Rechenaufwand bei dem Aufruf 
der Funktion calculate_hough_cirles() zu reduzieren.
Diese Datei enthält wenige Bilder mit meist gut sichtbaren Verkehrszeichen 

"""
### TO DO ###
# Definiere den Pfad zur gt.txt-Datei / zur new_gt.txt-Datei
GT_TXT_PATH = "./new_gt.txt"
assert os.path.exists(GT_TXT_PATH), "Der angegebene Pfad existriert nicht."

In [5]:
# Prohibitory Class IDs 
PROHIBITORY_CLASS_IDs = [ 0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 15, 16]

# Mandatory Class IDs
MANDATORY_CLASS_IDs = [ 33, 34, 35, 36, 37, 38, 39, 40 ]

# Danger Class IDs
DANGER_CLASS_IDs = [ 11, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31 ]

In [6]:
# MANDATORY-Dictionary 
MANDATORY_DICT = {}
MANDATORY_DICT_CLASS = {}
# MANDATORY images (filenames) list
MANDATORY_IMG_LIST = []
# MANDATORY filepaths
MANDATORY_FILEPATHS = []

## Aufgabe 1 - Aussortieren bestimmter Verkehrsschilder

In [7]:
def calculate_mandatory(): 
    """
    Sortiert Verkehrszeichen nach der Kategorie „mandatory“ und speichert 
    die Ergebnisse in ein Dictionary.
    Das Dictionary beinhaltet Dateinamen als Schlüssel und Listen von 
    Ground Thruth ROIs-Listen als Values.
    """
    
    # Setze prohibitory dictionary zurück
    MANDATORY_DICT.clear()
    
    # Öffne die gt.txt-Datei / new_gt.txt-Datei
    with open(GT_TXT_PATH, newline='') as csvfile:
        gt_reader = csv.reader(csvfile, delimiter=';')
        
        # Bau eine Schleife, um die Daten Zeile für Zeile auszulesen 
        # und die entsprechende Liste der ROIs für die Datei zu füllen
        for row in gt_reader:
            if int(row[5]) in MANDATORY_CLASS_IDs:
                roi = [int(row[1]), int(row[2]), int(row[3]), int(row[4])] # int(row[5]) dazu
                if str(row[0]) not in MANDATORY_DICT.keys(): 
                    MANDATORY_DICT[row[0]] = [roi]
                    MANDATORY_DICT_CLASS[row[0]] = [int(row[5])]
                else:
                    MANDATORY_DICT[row[0]].append(roi)
                    MANDATORY_DICT_CLASS[row[0]].append(int(row[5]))
        


In [8]:
# Funktionsaufruf
"""
Erwartete Ausgabe:
    {'00002.ppm': [[892, 476, 1006, 592, 39]],
     '00039.ppm': [[953, 252, 1015, 313, 35]],
     '00040.ppm': [[1040, 310, 1130, 400, 33]],
     '00066.ppm': [[569, 483, 618, 535, 38]],
     '00074.ppm': [[383, 527, 421, 569, 38]],
     '00081.ppm': [[233, 499, 249, 521, 38]],
     ...}
"""
calculate_mandatory()
print(MANDATORY_DICT)

{'00002.ppm': [[892, 476, 1006, 592]], '00039.ppm': [[953, 252, 1015, 313]], '00040.ppm': [[1040, 310, 1130, 400]], '00066.ppm': [[569, 483, 618, 535]], '00074.ppm': [[383, 527, 421, 569]], '00081.ppm': [[233, 499, 249, 521]], '00083.ppm': [[1071, 297, 1142, 368]], '00115.ppm': [[330, 523, 351, 545]], '00163.ppm': [[753, 473, 825, 545]], '00214.ppm': [[1214, 241, 1288, 312]], '00255.ppm': [[183, 503, 260, 581]], '00262.ppm': [[894, 282, 953, 341]], '00279.ppm': [[617, 484, 673, 540]], '00304.ppm': [[504, 385, 553, 434]], '00410.ppm': [[367, 615, 423, 671]], '00411.ppm': [[185, 602, 238, 662]], '00491.ppm': [[430, 568, 488, 637]], '00501.ppm': [[107, 637, 216, 748]], '00503.ppm': [[354, 579, 423, 650]], '00556.ppm': [[1195, 499, 1253, 557]], '00612.ppm': [[127, 521, 218, 612]], '00624.ppm': [[387, 505, 434, 554]], '00655.ppm': [[544, 520, 564, 540]], '00679.ppm': [[720, 409, 759, 450]], '00682.ppm': [[278, 426, 319, 467], [885, 421, 930, 466], [261, 486, 290, 515]], '00685.ppm': [[615, 

In [9]:
# Ermittele die Dateienamen (ausgehend von DATA_PATH) alle Treffer in MANDATORY_DICT
# MANDATORY_FILEPATHS = ...
MANDATORY_FILEPATHS= [os.path.join(DATA_PATH, img) for img in MANDATORY_DICT.keys()]
print(len(MANDATORY_FILEPATHS))

37


In [10]:
def map_int_to_sign_name(path_to_data_folder, csv_mapping_file):
    """
    Ordnet int-Zahl dem Schildnamen zu
    Argumente: Pfad zum Datenordner, Name der csv-Datei
    Rückgabe:  pandas-DataFrame aus der generierten csv-Datei und 
              eine Dictionary (dict) mit ClassIDs als keys und Traffic Sign Names als values
    """
    
    ###   TO DO   ###
    csv_path = csv_mapping_file
    
    assert os.path.exists(csv_path), "Der angegebene Pfad existriert nicht."
    # dict_mapping-Variable soll eine Dictionary (dict) sein, mit ClassIDs als keys und Traffic Sign Names als values
    dict_mapping = dict()

    with open(os.path.join(path_to_data_folder,"ReadMe.txt"), 'r') as f:
        for line in f:
            if len(line) > 4 and "=" in line:
                tmp = [int(i) for i in line.strip().split(" ")[0] if i.isdigit()]
                num = 0
                if len(tmp)>1:
                    for i in range(len(tmp)):
                        num+= 10**(len(tmp)-i-1) * tmp[i]
                elif tmp:
                    num = tmp[-1]
                if f"{num} = " in line:
                    dict_mapping[num] = line.split("=")[-1][:-1]

    return dict_mapping

In [11]:
DICT_MAPPING = map_int_to_sign_name(DATA_PATH, GT_TXT_PATH)

In [12]:
print(DICT_MAPPING)

{0: ' speed limit 20 (prohibitory)', 1: ' speed limit 30 (prohibitory)', 2: ' speed limit 50 (prohibitory)', 3: ' speed limit 60 (prohibitory)', 4: ' speed limit 70 (prohibitory)', 5: ' speed limit 80 (prohibitory)', 6: ' restriction ends 80 (other)', 7: ' speed limit 100 (prohibitory)', 8: ' speed limit 120 (prohibitory)', 9: ' no overtaking (prohibitory)', 10: ' no overtaking (trucks) (prohibitory)', 11: ' priority at next intersection (danger)', 12: ' priority road (other)', 13: ' give way (other)', 14: ' stop (other)', 15: ' no traffic both ways (prohibitory)', 16: ' no trucks (prohibitory)', 17: ' no entry (other)', 18: ' danger (danger)', 19: ' bend left (danger)', 20: ' bend right (danger)', 21: ' bend (danger)', 22: ' uneven road (danger)', 23: ' slippery road (danger)', 24: ' road narrows (danger)', 25: ' construction (danger)', 26: ' traffic signal (danger)', 27: ' pedestrian crossing (danger)', 28: ' school crossing (danger)', 29: ' cycles crossing (danger)', 30: ' snow (dan

In [13]:
def render_mandatory_rois():
    """
    Malt die ROIs (Rechtecke) auf die entsprechenden Bilder und speichert 
    die in MANDATORY_IMG_LIST. 
    Hinweis:
    Die ROIs und Bildernamen können aus MANDATORY_DICT ermittelt werden
    """
    # Setze die globale variable zurück
    MANDATORY_IMG_LIST.clear()
    
    for key in MANDATORY_DICT.keys():
        file_path = os.path.join(DATA_PATH, key)
        img = plt.imread(file_path)
        ### TO DO ###
        for idx in range(len(MANDATORY_DICT[key])):
            # print(len(MANDATORY_DICT[key]), idx)
            # Berechne Koordinaten des Rechtecks
            point1 = (MANDATORY_DICT[key][idx][0], MANDATORY_DICT[key][idx][1])
            point2 = (MANDATORY_DICT[key][idx][2], MANDATORY_DICT[key][idx][3])
            
             # Zeichne das Rechteck
            color = (255, 0, 0)
            thickness = 2
            # Zeichne das Rechteck mit Hilfe der berechneten Koordinaten in das Bild (cv2.rectangle) ein
            img = cv2.rectangle(img, pt1=point1, pt2=point2,color=color,thickness=thickness)
            
            # Speichere Verkehrszeichennamen als text
            org = (MANDATORY_DICT[key][idx][2] + 10, MANDATORY_DICT[key][idx][1] + 20)
            font = cv2.FONT_HERSHEY_SIMPLEX
            fontScale = 1
            # Speichere Verkehrszeichen-ID als string
            text = f"{DICT_MAPPING[MANDATORY_DICT_CLASS[key][idx]]}"
            # Nutze cv2.putText um die Verkehrszeichen-ID neben dem entsprechenden Verkehrszeichen zu positionieren
            img = cv2.putText(img, text,org, font, 
                    fontScale, color, thickness, cv2.LINE_AA)
            
        MANDATORY_IMG_LIST.append(img)

In [14]:
# Mandatory Image list abrufen
render_mandatory_rois()
print(len(MANDATORY_IMG_LIST))

37


In [31]:
def show_img(idx):
    plt.figure(figsize=(16,8))
    plt.imshow(MANDATORY_IMG_LIST[idx])
    plt.show()

In [32]:
interact(show_img, idx=widgets.IntSlider(min=0,max=len(MANDATORY_IMG_LIST)-1, step=1, value=0));

interactive(children=(IntSlider(value=0, description='idx', max=36), Output()), _dom_classes=('widget-interact…

## Aufgabe 2 – Formbasierter Ansatz


Hier kannst du die Links abrufen, die zum Implementieren dieser Funktion nützlich sein können:

- https://docs.opencv.org/4.x/da/d53/tutorial_py_houghcircles.html
- https://docs.opencv.org/4.x/df/d9d/tutorial_py_colorspaces.html
- https://docs.opencv.org/4.x/d4/d13/tutorial_py_filtering.html 

Übersicht der Tutorials: https://docs.opencv.org/4.x/d6/d00/tutorial_py_root.html

In [15]:
def calculate_hough_cirles(filepaths, d_p, min_dist, param1, param2, min_radius, max_radius):
    """
     Berechnet Hough Circles unter Berücksichtigung der Form der Verkehrszeichen
    """
    
    # Liste fuer die Speicherung des Ergebnis
    result = []
    predicted_dict = {}
    
    for filepath_ in filepaths:
        # Lade das Bild in color_img
        color_img = cv2.imread(filepath_, cv2.IMREAD_COLOR)
        ### TO DO ###
        
        # Konvertiere das BGR-Bild in Gray.
        # https://docs.opencv.org/4.x/df/d9d/tutorial_py_colorspaces.html
        img_gray = cv2.cvtColor(color_img,cv2.COLOR_BGR2GRAY)
       
        # Reduziere das Rauschen 
        # https://docs.opencv.org/4.x/d4/d13/tutorial_py_filtering.html --> Kapitel "Image bluring"
        img_blurred = cv2.medianBlur(img_gray,5)
        
        # Ermittele die Kreisen auf dem Bild 
        # https://docs.opencv.org/4.x/da/d53/tutorial_py_houghcircles.html
        circles = cv2.HoughCircles(img_blurred,cv2.HOUGH_GRADIENT,d_p, min_dist,
                            param1=param1,param2=param2,minRadius=min_radius,maxRadius=max_radius)
        # Kreise auf das Bild malen
        if circles is not None: 
            # Kreise-Paramater in interger umwandeln
            circles = np.uint16(np.around(circles)) 
            key = filepath_.split("/")[-1]
            # Kreise auf das Bild malen
            for idx, point in enumerate(circles[0, :]): 
                a, b, r = point[0], point[1], point[2]
                color_img = cv2.circle(color_img, (a, b), r, (0,255,0),2)
                # Ermittle Koordinaten der Rechtecke, die für die Evaluation benutzt werden
                point1 = (int(max(0,a-r)), int(max(0,b-r)))
                point2 = (int(a+r), int(b+r))
                # OPTIONAL: Rechtecke auf das Bild malen
                color_img =  cv2.rectangle(color_img, pt1=point1, pt2=point2,color=(255,0,0),thickness=2)
                if os.path.split(filepath_)[-1] in predicted_dict:
                    if a==0 and b==0 and r==0:
                        continue
                    predicted_dict[os.path.split(filepath_)[-1]].append([point1[0], point1[1], point2[0], point2[1]])
                else: 
                    predicted_dict[os.path.split(filepath_)[-1]] = []
                    if a==0 and b==0 and r==0:
                        continue
                    predicted_dict[os.path.split(filepath_)[-1]].append([point1[0], point1[1], point2[0], point2[1]])
        else:
            predicted_dict[os.path.split(filepath_)[-1]] = []
        result.append(color_img)
        
    return result, predicted_dict

In [86]:
pred_imgs_form, predicted_rect_rois = calculate_hough_cirles(MANDATORY_FILEPATHS, 
                                                  d_p=1.5,
                                                  min_dist=30, 
                                                  param1=90, 
                                                  param2=100, 
                                                  min_radius=10, # 20
                                                  max_radius=60) # 120

print(len(predicted_rect_rois.keys()))

37


In [84]:
def show_img_form(idx):
    plt.figure(figsize=(16,8))
    plt.imshow(cv2.cvtColor(pred_imgs_form[idx], cv2.COLOR_BGR2RGB))
    plt.show()

In [85]:
interact(show_img_form, idx=widgets.IntSlider(min=0, max=len(pred_imgs_form)-1, step=1, value=0));

interactive(children=(IntSlider(value=0, description='idx', max=9), Output()), _dom_classes=('widget-interact'…

## Aufgabe 3 – Optimierung und Evaluation des formbasierten Ansatzes

In [87]:
def jaccard_similarity(pred, gr_truth):
    '''
    Berechnet den Jaccard-Koeffizienten für zwei Rechtecke: den vorhergesagen (pred) und den ground_truth (gr_truth)
    
    '''
    # Ermittle die (x, y)-Koordinaten der Schnittmenge beider Rechtecke
    x_i1 = max(pred[0], gr_truth[0])
    y_i1 = max(pred[1], gr_truth[1])
    x_i2 = min(pred[2], gr_truth[2])
    y_i2 = min(pred[3], gr_truth[3])

    inter_area = max(0, x_i2 - x_i1 + 1) * max(0, y_i2 - y_i1 + 1)

    pred_area = (pred[2] - pred[0] + 1) * (pred[3] - pred[1] + 1)
    gr_truth_area = (gr_truth[2] - gr_truth[0] + 1) * (gr_truth[3] - gr_truth[1] + 1)
    
    iou = inter_area / float(pred_area + gr_truth_area - inter_area)
    
    # Gebe den "Intersection Over Union"-Wert zurück
    return iou

In [88]:
def evaluate_detection(ground_truth_dict, predicted_dict, similarity_threshold=0.6):    
    '''
    Evaluiert implementierte Ansätze anhand des Jaccard-Ähnlichkeitsmaßes
    Referenz für die Berechnung: Houben et. al. Kapitel IV Evaluation Procedure
    '''
    # True Positives
    tp = 0
    # False Positives
    fp = 0
    # False Negatives
    fn = 0
    
    for key in ground_truth_dict.keys():
        # Liste mit allen ROIs eines Dateinamens 
        rois_gt_lists = ground_truth_dict[key]
        
        # Berechne Jaccard-Ähnlichkeitsmaß von detektierten Rechtecken, die aus den Kreiskoordinaten ermittelt wurden
        rois_pred_lists = predicted_dict[key]
        
        if len(rois_pred_lists) > 0:
            for rois_gt_list in rois_gt_lists:
                iou = [jaccard_similarity(rois_pred, rois_gt_list) for rois_pred in rois_pred_lists]
                
                # Liste mit den Werten, die kleiner als similarity_threshold sind
                iou_lt_threshold = [value for value in iou if value < similarity_threshold]
                fp = fp + len(iou_lt_threshold)
                
                # Liste mit den Werten, die größer / gleich similarity_threshold sind
                iou_gt_threshold = [value for value in iou if value >= similarity_threshold]
                
                if len(iou_gt_threshold) > 0 : 
                    tp = tp + 1
                else:
                    fn = fn + 1
        else:
            fn = fn + len(rois_gt_lists)

    return tp, fp, fn            

In [92]:
def calculate_precision_recall(tp, fp, fn):
    '''
    Berechnet Precision- und Recall-Werte
    '''
    
    precision = math.nan
    if tp + fp != 0:
        precision = tp / (tp + fp)   
    
    recall = math.nan
    if tp + fn != 0:
        recall = tp / (tp + fn)
        
    return precision, recall

In [97]:
# similarity_threshold entspricht dem Schwellenwert im Paper von Houben et. al.
tp_form, fp_form, fn_form = evaluate_detection(MANDATORY_DICT, predicted_rect_rois, similarity_threshold=0.6)

TypeError: object of type 'int' has no len()

In [96]:
# Precision-Recall-Plot
precision, recall = calculate_precision_recall(tp_form, fp_form, fn_form)


TypeError: object of type 'float' has no len()

In [98]:
print(precision, recall)

0.10256410256410256 0.10256410256410256
